<a href="https://colab.research.google.com/github/jacobeturpin/CXR14/blob/master/CRX14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Thorax Diseases Using the ChestX-Ray14 Dataset and Convolutional Techniques

## Introduction

Dataset provided by the National Institute of Health at: https://nihcc.app.box.com/v/ChestXray-NIHCC

*Random subset provided [here](https://www.kaggle.com/nih-chest-xrays/sample)*

## Setup

### Environment

In [1]:
# Expecting "python3" env to be the active env on SageMaker Notebook instances
!conda info --envs

# conda environments:
#
base                     /home/ec2-user/anaconda3
JupyterSystemEnv         /home/ec2-user/anaconda3/envs/JupyterSystemEnv
R                        /home/ec2-user/anaconda3/envs/R
amazonei_mxnet_p27       /home/ec2-user/anaconda3/envs/amazonei_mxnet_p27
amazonei_mxnet_p36       /home/ec2-user/anaconda3/envs/amazonei_mxnet_p36
amazonei_tensorflow_p27     /home/ec2-user/anaconda3/envs/amazonei_tensorflow_p27
amazonei_tensorflow_p36     /home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36
chainer_p27              /home/ec2-user/anaconda3/envs/chainer_p27
chainer_p36              /home/ec2-user/anaconda3/envs/chainer_p36
mxnet_p27                /home/ec2-user/anaconda3/envs/mxnet_p27
mxnet_p36                /home/ec2-user/anaconda3/envs/mxnet_p36
python2                  /home/ec2-user/anaconda3/envs/python2
python3               *  /home/ec2-user/anaconda3/envs/python3
pytorch_p27              /home/ec2-user/anaconda3/envs/pytorch_p27
pytorch_p36              /ho

In [2]:
!pip install keras

You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
!conda install -n python3 tensorflow-gpu --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/python3

  added / updated specs: 
    - tensorflow-gpu


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cudatoolkit-10.0.130       |                0       380.0 MB
    scipy-1.4.1                |   py36h0b6359f_0        18.9 MB
    cudnn-7.6.5                |       cuda10.0_0       226.4 MB
    tensorflow-base-2.0.0      |gpu_py36h0ec5d1f_0       329.9 MB
    _tflow_select-2.1.0        |              gpu           2 KB
    joblib-0.14.1              |             py_0         202 KB
    scikit-learn-0.22.1        |   py36hd81dba3_0         7.1 MB
    mkl-2020.0                 |              166       202

In [8]:
!conda upgrade scikit-image --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/python3

  added / updated specs: 
    - scikit-image


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    scikit-image-0.16.2        |   py36h0573a6f_0        25.0 MB

The following packages will be UPDATED:

    scikit-image: 0.13.1-py36h14c3975_1 --> 0.16.2-py36h0573a6f_0


scikit-image-0.16.2  | 25.0 MB   | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [4]:
!pip install efficientnet --quiet

You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!conda list -n python3

# packages in environment at /home/ec2-user/anaconda3/envs/python3:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
_tflow_select             2.1.0                       gpu  
absl-py                   0.9.0                    py36_0  
alabaster                 0.7.10           py36h306e16b_0  
anaconda-client           1.6.14                   py36_0  
anaconda-project          0.8.2            py36h44fb852_0  
argparse                  1.4.0                     <pip>
asn1crypto                0.24.0                   py36_0  
astor                     0.8.0                    py36_0  
astroid                   1.6.3                    py36_0  
astropy                   3.0.2            py36h3010b51_1  
attrs                     18.1.0                   py36_0  
Automat                   0.3.0                     <pip>
autovizwidget             0.13.1                    <pip>
awscli                    1.1

In [6]:
!conda env export -n python3 --no-builds

name: python3
channels:
  - conda-forge
  - defaults
dependencies:
  - ffmpeg=4.0.2
  - freeglut=3.0.0
  - gnutls=3.5.19
  - libgfortran=3.0.0
  - libglu=9.0.0
  - libiconv=1.15
  - libprotobuf=3.6.1
  - mpi=1.0
  - nb_conda=2.2.1
  - nb_conda_kernels=2.2.2
  - nettle=3.3
  - openh264=1.8.0
  - openmpi=3.1.0
  - protobuf=3.6.1
  - requests=2.20.0
  - x264=1!152.20180806
  - xorg-fixesproto=5.0
  - xorg-inputproto=2.3.2
  - xorg-kbproto=1.0.7
  - xorg-libx11=1.6.9
  - xorg-libxau=1.0.9
  - xorg-libxext=1.3.4
  - xorg-libxfixes=5.0.3
  - xorg-libxi=1.7.10
  - xorg-xextproto=7.3.0
  - xorg-xproto=7.0.31
  - _libgcc_mutex=0.1
  - _tflow_select=2.1.0
  - absl-py=0.9.0
  - alabaster=0.7.10
  - anaconda-client=1.6.14
  - anaconda-project=0.8.2
  - asn1crypto=0.24.0
  - astor=0.8.0
  - astroid=1.6.3
  - astropy=3.0.2
  - attrs=18.1.0
  - babel=2.5.3
  - backcall=0.1.0
  - backports=1.0
  - backports.shutil_get_terminal_size=1.0.0
  - beautifulsoup4=4.6.0
  - bitarray=0.8.1
  - bkcharts=0.2
  -

### Imports and Constants

In [3]:
import glob
import gzip
import os
import tarfile
import time
import warnings
from urllib.request import urlretrieve

import pandas as pd

import keras
from keras.applications import DenseNet121, ResNet50
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Flatten
from keras.metrics import AUC
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

import efficientnet.keras as efn

Using TensorFlow backend.


In [4]:
import tensorflow as tf
tf.test.is_gpu_available()

True

In [5]:
# Change to False to force local file system usage
USE_DRIVE = False

if USE_DRIVE:

    try:
        from google.colab import drive
        USE_DRIVE = True
        ROOT_DIR = './drive/My Drive'
    except:
        warnings.warn('Google Drive Not Found -- Using local file system')
        USE_DRIVE = False
        ROOT_DIR = '.'
else:
    ROOT_DIR = '.'

In [ ]:
DATA_PATH = '/CXR14'
CHECKPOINT_PATH = '/models'

SAMPLE_RATE = 0.15
EPOCHS = 2
BATCH_SIZE = 64
CHECKPOINT_RATE = 2

CLASSES = [
  'Hernia',
  'Pneumonia',
  'Fibrosis',
  'Edema',
  'Emphysema',
  'Cardiomegaly',
  'Pleural_Thickening',
  'Consolidation',
  'Pneumothorax',
  'Mass',
  'Nodule',
  'Atelectasis',
  'Effusion',
  'Infiltration',
  'No Finding'
]

## Data Loading

In [ ]:
# Determines whether to use random sample (approx. 3K files) or full >120K files dataset
USE_SAMPLE = False

In [ ]:
if USE_DRIVE:
    drive.mount('/content/drive')

In [ ]:
def batch_download_and_extract(path='.', first_n=None):

    # URLs for zip files containing ChestX-ray14 dataset from NIH
    links = [
        'https://nihcc.box.com/shared/static/vfk49d74nhbxq3nqjg0900w5nvkorp5c.gz',
        'https://nihcc.box.com/shared/static/i28rlmbvmfjbl8p2n3ril0pptcmcu9d1.gz',
        'https://nihcc.box.com/shared/static/f1t00wrtdk94satdfb9olcolqx20z2jp.gz',
        'https://nihcc.box.com/shared/static/0aowwzs5lhjrceb3qp67ahp0rd1l1etg.gz',
        'https://nihcc.box.com/shared/static/v5e3goj22zr6h8tzualxfsqlqaygfbsn.gz',
        'https://nihcc.box.com/shared/static/asi7ikud9jwnkrnkj99jnpfkjdes7l6l.gz',
        'https://nihcc.box.com/shared/static/jn1b4mw4n6lnh74ovmcjb8y48h8xj07n.gz',
        'https://nihcc.box.com/shared/static/tvpxmn7qyrgl0w8wfh9kqfjskv6nmm1j.gz',
        'https://nihcc.box.com/shared/static/upyy3ml7qdumlgk2rfcvlb9k6gvqq2pj.gz',
        'https://nihcc.box.com/shared/static/l6nilvfa9cg3s28tqv1qc1olm3gnz54p.gz',
        'https://nihcc.box.com/shared/static/hhq8fkdgvcari67vfhs7ppg2w6ni4jze.gz',
        'https://nihcc.box.com/shared/static/ioqwiy20ihqwyr8pf4c24eazhh281pbu.gz'
    ]

    first_n = first_n or len(links)

    if first_n > len(links):
        raise('Number of files requested exceeds amount available')

    for idx, link in enumerate(links[:first_n]):
        fn = 'images_{:03d}.tar.gz'.format(idx+1)
        print('downloading', fn, '...')
        urlretrieve(link, fn)  # download the zip file

        tar = tarfile.open(fn, "r:gz")
        tar.extractall(path + '/images_{:03d}'.format(idx+1))
        tar.close()

        os.remove(fn)  # Remove remaining .tar file

    labels_url = 'https://nihcc.app.box.com/index.php?rm=box_download_shared_file&vanity_name=ChestXray-NIHCC&file_id=f_219760887468'
    urlretrieve(labels_url, path + '/Data_Entry_2017.csv')
  
    print("Download complete. Please check the checksums")

In [ ]:
full_dir = "{0}{1}/full".format(ROOT_DIR, DATA_PATH)

if not os.path.isdir(full_dir):
    print('Data not present -- downloading now ...')
    os.makedirs(full_dir)
    batch_download_and_extract(full_dir)
else:
    print('Data directory already exists')

In [ ]:
df = pd.read_csv("{}/Data_Entry_2017.csv".format(full_dir))
df.head()

In [ ]:
df['Finding Labels'] = df['Finding Labels'].apply(lambda s: s.split('|'))
df.head()

In [ ]:
# https://stackoverflow.com/questions/45312377/how-to-one-hot-encode-from-a-pandas-column-containing-a-list

mlb = MultiLabelBinarizer()
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('Finding Labels')),
                          columns=mlb.classes_,
                          index=df.index))
df.head()

In [ ]:
df = df[['Image Index'] + CLASSES]
df.head()

In [ ]:
full_dir

In [ ]:
img_paths =  glob.glob(full_dir + '/**/*.png', recursive=True)
img_paths[:5]

In [ ]:
df['Image Index'] = df['Image Index'].apply(lambda x: next(p for p in img_paths if x in p))
df.head()

In [ ]:
#df = df.sample(frac=SAMPLE_RATE)

## Data Preparation

The ChestX-ray14 dataset is too large to fit entirely in memory when training; therefore, it's incrementally loaded via generator to reduce memory overhead. This is achieved using the Keras [Image Proprocessing](https://keras.io/preprocessing/image/) submodule.

In [ ]:
# https://datascience.stackexchange.com/a/17445/91316

train_df, test_df = train_test_split(df, test_size=0.2)

print('Training/Validation Samples:  {}'.format(len(train_df)))
print('Test Samples:  {}'.format(len(test_df)))

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.25
)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col='Image Index',
    y_col=CLASSES,
    subset='training',
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='raw',
    #classes=CLASSES,
    target_size=(224, 224)
)

In [ ]:
valid_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col='Image Index',
    y_col=CLASSES,
    subset='validation',
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='raw',
    #classes=[],
    target_size=(224, 224)
)

In [ ]:
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=None,
    x_col='Image Index',
    y_col=CLASSES,
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='raw',
    #classes=[],
    target_size=(224, 224)
)

## Modeling

In [ ]:
full_dir + CHECKPOINT_PATH

In [ ]:
if not os.path.exists(full_dir + CHECKPOINT_PATH):
    os.makedirs(full_dir + CHECKPOINT_PATH)

In [ ]:
class TimeHistory(keras.callbacks.Callback):
    """Object used on keras callbacks to measure epoch training time

    Args:
        None

    Params:
        time (list): collection of times in seconds for each epoch's training

    """

    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

Three models will be implemented and their results compared:

1.   ResNet
2.   DenseNet
3.   EfficientNet


### ResNet

A pre-built ResNet model from the Keras library is used. Documentation on the model can be found [here](https://keras.io/applications/). Pre-trained weights from the ImageNet dataset are used.

In [32]:
resnet_base = ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
    pooling='avg'
)
output = Dense(15, activation='sigmoid')(resnet_base.output)

resnet = Model(input=resnet_base.input, outputs=output)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


In [33]:
resnet.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [34]:
resnet.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['binary_accuracy', tf.keras.metrics.AUC(multi_label=True)]
)

In [35]:
resnet_time = TimeHistory()
resnet_stopping = EarlyStopping(patience=5, restore_best_weights=True)
resnet_checkpoint = ModelCheckpoint(filepath=full_dir + CHECKPOINT_PATH + 'resnet-{epoch:03d}.hdf5', 
                                    period=CHECKPOINT_RATE)

resnet_history = resnet.fit_generator(
    generator=train_generator,
    epochs=EPOCHS,
    shuffle=True,
    validation_data=valid_generator,
    callbacks=[resnet_time, resnet_stopping, resnet_checkpoint]
)

Epoch 1/2
1052/1052 [==============================] - 1378s 1s/step - loss: 0.1968 - binary_accuracy: 0.9299 - auc_1: 0.8767 - val_loss: 0.3393 - val_binary_accuracy: 0.9226 - val_auc_1: 0.8107
Epoch 2/2
1052/1052 [==============================] - 1324s 1s/step - loss: 0.1867 - binary_accuracy: 0.9323 - auc_1: 0.8939 - val_loss: 0.1857 - val_binary_accuracy: 0.9311 - val_auc_1: 0.8859


### DenseNet

In [37]:
densenet_base = DenseNet121(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
    pooling='avg'
)
output = Dense(15, activation='sigmoid')(densenet_base.output)

densenet = Model(input=densenet_base.input, outputs=output)

29089792/29084464 [==============================] - 0s 0us/step


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


In [38]:
densenet.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

In [41]:
densenet.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['binary_accuracy', AUC()]
)

In [42]:
densenet_time = TimeHistory()
densenet_stopping = EarlyStopping(patience=5, restore_best_weights=True)
densenet_checkpoint = ModelCheckpoint(filepath=full_dir + CHECKPOINT_PATH + 'densenet-{epoch:03d}.hdf5', 
                                      period=CHECKPOINT_RATE)

densenet.fit_generator(
    generator=train_generator,
    epochs=EPOCHS,
    shuffle=True,
    validation_data=valid_generator,
    callbacks=[densenet_time, densenet_stopping, densenet_checkpoint]
)

Epoch 1/2
1052/1052 [==============================] - 1445s 1s/step - loss: 0.1964 - binary_accuracy: 0.9296 - auc_2: 0.8785 - val_loss: 0.2293 - val_binary_accuracy: 0.9285 - val_auc_2: 0.8617
Epoch 2/2
1052/1052 [==============================] - 1377s 1s/step - loss: 0.1854 - binary_accuracy: 0.9322 - auc_2: 0.8963 - val_loss: 0.1697 - val_binary_accuracy: 0.9186 - val_auc_2: 0.8577


### EfficientNet

EfficientNet is a lightweight CNN architecture that is designed to require significantly less compute than other state of the art architectures on common transfer learning datasets.

Pre-built EfficientNet models built in Keras are used from the efficientnet library available on [GitHub](https://github.com/qubvel/efficientnet) and installable via PyPI.

In [ ]:
efficientnet_base = efn.EfficientNetB4(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
    pooling='avg'
)
output = Dense(15, activation='sigmoid')(efficientnet_base.output)

efficientnet = Model(input=efficientnet_base.input, outputs=output)

In [ ]:
efficientnet.summary()

In [ ]:
efficientnet.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['binary_accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

In [ ]:
efficientnet_time = TimeHistory()
efficientnet_stopping = EarlyStopping(patience=5, restore_best_weights=True)
efficientnet_checkpoint = ModelCheckpoint(filepath=full_dir + CHECKPOINT_PATH + 'efficientnet-{epoch:03d}.hdf5', 
                                          period=CHECKPOINT_RATE)

efficientnet_history = efficientnet.fit_generator(
    generator=train_generator,
    epochs=EPOCHS,
    shuffle=True,
    validation_data=valid_generator,
    callbacks=[efficientnet_time, efficientnet_stopping, efficientnet_checkpoint]
)

## Results

### ResNet

In [ ]:
resnet_pred = resnet.predict_generator(
    generator=test_generator
)

In [ ]:
for idx, cls in enumerate(CLASSES):
  print('{} AUC:  '.format(cls), roc_auc_score(test_df[cls], resnet_pred[:,idx]))

### DenseNet

In [ ]:
densenet_pred = densenet.predict_generator(
    generator=test_generator
)

In [ ]:
for idx, cls in enumerate(CLASSES):
  print('{} AUC:  '.format(cls), roc_auc_score(test_df[cls], densenet_pred[:,idx]))

### EfficientNet

In [ ]:
efficientnet_pred = efficientnet.predict_generator(
    generator=test_generator
)

In [ ]:
for idx, cls in enumerate(CLASSES):
  print('{} AUC:  '.format(cls), roc_auc_score(test_df[cls], efficientnet_pred[:,idx]))

## Conclusions